In [1]:
import pandas as pd
import numpy as np
labels = pd.read_csv("/Users/mingliangang/Desktop/cifar10/trainLabels.csv")
labels.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [2]:
label2id = {x:i for i,x in enumerate(list(set(labels["label"])))}
id2label = {i:x for i,x in enumerate(list(set(labels["label"])))}
labelcon = lambda x: label2id[x]
labels['label'] = labels['label'].map(labelcon)
labels_ = labels['label'].as_matrix()

In [3]:
labels['label'].head()

0    3
1    8
2    8
3    2
4    5
Name: label, dtype: int64

In [4]:
labels_

array([3, 8, 8, ..., 8, 5, 5])

# Sanity check

In [5]:
labels['label'].map(lambda x: id2label[x]).head()

0          frog
1         truck
2         truck
3          deer
4    automobile
Name: label, dtype: object

# Tensorflow input pipeline

In [6]:
import os
from sklearn.model_selection import train_test_split
train = ["/Users/mingliangang/Desktop/cifar10/train/"+i for i in os.listdir("/Users/mingliangang/Desktop/cifar10/train")]
X_train, X_val, y_train, y_val = train_test_split(train, labels_, test_size=0.33, random_state=42)

In [7]:
import tensorflow as tf

NUM_CLASSES = 10 
def input_parser(img_path, label):
    # convert the label to one-hot encoding
    one_hot = tf.one_hot(label, NUM_CLASSES)

    # read the img from file
    img_file = tf.read_file(img_path)
    img_decoded = tf.image.decode_image(img_file, channels=3)

    return tf.cast(img_decoded,tf.float32), one_hot

In [10]:
tf.__version__

'1.3.0'

In [8]:
from tensorflow.contrib.data import Dataset
tr_data = Dataset.from_tensor_slices((X_train, y_train))

In [9]:
tr_data = tr_data.map(input_parser)

In [10]:
batch_size = 128
batched_dataset = tr_data.batch(batch_size)
iterator = batched_dataset.make_initializable_iterator()
next_element = iterator.get_next()

In [11]:
sess = tf.Session()

# Resnet Model

In [12]:
from models import resnet
X = tf.placeholder("float", [batch_size, 32, 32, 3])
Y = tf.placeholder("float", [batch_size, 10])
learning_rate = tf.placeholder("float", [])
net = resnet(X,20)
cross_entropy = -tf.reduce_sum(Y*tf.log(net))
opt = tf.train.MomentumOptimizer(learning_rate, 0.9,use_nesterov=True)
train_op = opt.minimize(cross_entropy)

# with tf.device('/device:GPU:0'):
#     X = tf.placeholder("float", [batch_size, 32, 32, 3])
#     Y = tf.placeholder("float", [batch_size, 10])
#     learning_rate = tf.placeholder("float", [])
#     net = resnet(X,20)
#     cross_entropy = -tf.reduce_sum(Y*tf.log(net))
#     opt = tf.train.MomentumOptimizer(learning_rate, 0.9,use_nesterov=True)
#     train_op = opt.minimize(cross_entropy)
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())



Tensor("Shape:0", shape=(4,), dtype=int32)


In [13]:
#Multiple epochs
n_epochs = 1
for _ in range(n_epochs ):
  sess.run(iterator.initializer)
  while True:
    try:
      #There must be a better way tensor -> numpy -> tensor it seems kinda of stupid, there must be a better way to pipe. 
      loss,_ = sess.run([cross_entropy,train_op], feed_dict={X: sess.run(next_element[0]), Y: sess.run(next_element[1]),learning_rate: 0.001})
      print(loss)
    except ValueError:
      break

297.543
294.721
296.94
296.021
295.627
295.157
294.753
294.933
294.199
294.026
295.399
296.732
294.382
295.115
296.814
296.134
294.432
294.428
295.675
295.061
295.217
295.639
294.306
296.607
295.74
293.285
297.183
295.338
295.439
296.409
294.795
294.694
293.149
294.017
294.718
295.856
293.735
296.767
295.484
294.968
296.8
295.216
294.853
295.463
296.018
295.279
296.282
295.435
294.649
294.745
294.789
295.014
295.112
293.367
293.047
296.938
295.318
293.513
295.324
296.531
296.805
294.503
295.352
294.923
293.891
295.346
295.507
295.893
294.284
296.971
297.693
295.312
295.486
296.156
296.964
294.633
295.123
294.712
295.232
294.673
296.024
294.739
295.271
295.467
295.052
295.488
294.932
295.434
294.394
293.964
294.283
294.19
295.331
293.871
296.783
294.621
294.776
296.061
295.852
294.645
294.491
295.603
295.854
295.58
294.288
295.774
294.774
294.777
294.769
295.037
295.221
294.06
295.805
293.578
292.654
294.314
294.908
295.206
295.828
297.185
296.664
294.597
294.954
295.465
294.829
295.159